In [1]:
import pandas as pd
import scipy.constants
from scipy.optimize import curve_fit
from scipy.integrate import quad
from sklearn.metrics import r2_score
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib.transforms as transforms
from matplotlib.ticker import LogFormatter
import os
import fnmatch
%matplotlib qt
# %matplotlib inline
import datetime as dt
pd.set_option('display.max_rows', 100) 

## Reading in the data and preparing DataFrames

#### MID

In [2]:
#read in the data
mid = pd.read_csv("MID.tsv", sep="\t", skiprows=lambda x: x <=11,  on_bad_lines="skip")
mid.drop([0,1], inplace=True)
print(mid.describe())
#bring the time data into a new df
time = pd.DataFrame(mid.iloc[:,0::2]) #- get every second column
print(time.describe())
#add elapsed time columns
def elapsed_time(df):
    itr = 0
    counter = len(df.columns)
    masses = [2,4,5.5,12,15,16,18,28,32,40,44]
    for i in df:
        timestamp = pd.to_datetime(df[i], format="%Y/%m/%d %H:%M:%S.%f")
        runtime = (timestamp-timestamp[2]).dt.total_seconds()
        colname = "Elapsed time, mass {}".format(masses[itr]) 
        df.insert(counter, colname, runtime)
        itr += 1
        counter += 1
    return(df)
elapsed_time(time)

time_sliced = time.iloc[:,11:].fillna(0)

#convert elapsed times in time-dataframe back to a timestamp
def h2_time(df1,df2):
    startdate = pd.to_datetime(df1.iloc[2,0],format="%Y/%m/%d %H:%M:%S.%f")
    dic= {}
    itr = 0
    masses = [2,4,5.5,12,15,16,18,28,32,40,44]
    key = "Timestamp, mass {}".format(masses[itr])
    for label,content in df2.items():
        key = "Timestamp, mass {}".format(masses[itr])
        #.strftime('%d-%m-%Y %H:%M:%S') not used
        for row in content:
            dic.setdefault(key,[]).append((startdate+dt.timedelta(milliseconds=row*1000)))
        itr += 1
    new_df = pd.DataFrame.from_dict(dic)
    return(new_df)

new_df = h2_time(time,time_sliced)
new_df


       mass 2.0_value  mass 12.0_value  mass 15.0_value  mass 18.0_value  \
count    4.119000e+03     4.119000e+03     3.280000e+03     1.710000e+03   
mean     8.394687e-11     3.860746e-13     5.354454e-14     5.744327e-14   
std      1.433451e-10     2.674586e-13     7.174031e-14     1.402723e-13   
min      9.314800e-12     0.000000e+00     0.000000e+00     0.000000e+00   
25%      1.766485e-11     1.561500e-13     9.100000e-15     8.100000e-15   
50%      3.507500e-11     3.316000e-13     2.645000e-14     1.950000e-14   
75%      8.727315e-11     6.007000e-13     6.070000e-14     4.090000e-14   
max      1.098640e-09     9.504000e-13     4.825000e-13     1.080800e-12   

       mass 28.0_value  mass 32.0_value  mass 44.0_time  mass 44.0_value  
count     3.800000e+01     2.400000e+01             0.0              0.0  
mean      7.767816e-13     3.818750e-14             NaN              NaN  
std       4.960037e-13     4.206864e-14             NaN              NaN  
min       1.000

,"Timestamp, mass 2","Timestamp, mass 4","Timestamp, mass 5.5"
0,2023-02-24 11:37:08.273,2023-02-24 11:37:08.273,2023-02-24 11:37:08.273
1,2023-02-24 11:37:13.246,2023-02-24 11:37:13.246,2023-02-24 11:37:08.273
2,2023-02-24 11:37:18.219,2023-02-24 11:37:18.219,2023-02-24 11:37:08.273
3,2023-02-24 11:37:23.193,2023-02-24 11:37:23.193,2023-02-24 11:37:08.273
4,2023-02-24 11:37:28.166,2023-02-24 11:37:28.166,2023-02-24 11:37:08.273
...,...,...,...
4114,2023-02-24 11:37:08.273,2023-02-24 11:37:08.273,2023-02-24 11:37:08.273
4115,2023-02-24 11:37:08.273,2023-02-24 11:37:08.273,2023-02-24 11:37:08.273
4116,2023-02-24 11:37:08.273,2023-02-24 11:37:08.273,2023-02-24 11:37:08.273
4117,2023-02-24 11:37:08.273,2023-02-24 11:37:08.273,2023-02-24 11:37:08.273


#### Vaclogger

In [3]:
#reading the Emission current, pressures
vaclog=pd.read_csv("vaclog", sep="\t")
vaclog.head()
#adding an elapsed time column
#making the column into a datetime object
vac_timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(vac_timestamps-vac_timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
#drop index column (not necessary)
# vaclog.drop(index=vaclog.index[0], 
#         axis=0, 
#         inplace=True)
vaclog["Time"] = vac_timestamps.dt.strftime('%d-%m-%Y %H:%M:%S')
vaclog["Time"] = pd.to_datetime(vaclog["Time"],format='%d-%m-%Y %H:%M:%S')

vaclog.head()

,Live comments,Time,Elapsed time,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-platinum,T-CERNOX,I_emission,I_grid
0,NaN,2023-02-24 10:52:44,0.0,82.448,1.170000e-09,1.410000e-09,4.996000e-09,2.520000e-07,1.1,466.0,-5.050,4274.281,NaN,NaN
1,NaN,2023-02-24 10:52:51,7.0,82.448,1.170000e-09,1.410000e-09,4.996000e-09,2.529000e-07,1.1,466.0,-5.275,4274.281,NaN,NaN
2,NaN,2023-02-24 10:52:58,14.0,82.447,1.170000e-09,1.400000e-09,4.996000e-09,2.526000e-07,1.1,466.0,-5.275,4274.506,NaN,NaN
3,NaN,2023-02-24 10:53:04,20.0,82.447,1.170000e-09,1.410000e-09,4.996000e-09,2.526000e-07,1.1,466.0,-5.050,4274.618,NaN,NaN
4,NaN,2023-02-24 10:53:11,27.0,82.452,1.170000e-09,1.430000e-09,4.996000e-09,2.529000e-07,1.1,466.0,-5.050,4274.730,NaN,NaN


#### HiVolta log

In [4]:
#Reading in the HiVolta measurement file 
hv = pd.read_csv("hv log", sep=",")
#hv.drop(columns=["Vmon1","Vmon2","Vmon3","Vmon4","Vmon5","Vmon6","Vmon7","Vmon8","Imon3","Imon4","Imon5","Imon6","Imon7","Imon8","Comments1"], inplace=True)
hv["hv_grid"]=[element * 1e-6 for element in hv["IMon1"]]
hv["em"]=[element * 1e-6 for element in hv["IMon2"]]
hv.drop(hv.columns[hv.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
hv["em"]=abs(hv["em"])
hv["hv_grid"]=abs(hv["hv_grid"])

#adding a datetime column
datetime = pd.to_datetime(hv['Date'] + ' ' + hv['Time'],format="%d/%m/%Y %H:%M:%S.%f")
datetime = datetime.dt.strftime('%d-%m-%Y %H:%M:%S')
datetime = pd.to_datetime(datetime, format='%d-%m-%Y %H:%M:%S')
hv.insert(0,"datetime",datetime)
    
#making the column into a datetime object
hv_timestamps=pd.to_datetime(hv["Time"],format="%H:%M:%S.%f")
runtime=(hv_timestamps-hv_timestamps[0]).dt.total_seconds()    
hv.insert(2,"Elapsed time",runtime)

hv.head()



,datetime,Date,Elapsed time,Time,VMon1,VMon2,VMon3,VMon4,VMon5,VMon6,...,IMon2,IMon3,IMon4,IMon5,IMon6,IMon7,IMon8,Comment,hv_grid,em
0,2023-02-24 11:36:53,24/02/2023,0.000,11:36:53.726,0.0,0.08,0.04,0.12,0.0,0.0,...,-0.0197,-0.015,-0.0251,-0.0148,0.0142,-0.0066,-0.0157,NaN,8.010000e-08,1.970000e-08
1,2023-02-24 11:36:54,24/02/2023,1.168,11:36:54.894,0.0,0.08,0.04,0.12,0.0,0.0,...,-0.0197,-0.015,-0.0251,-0.0148,0.0142,-0.0066,-0.0157,NaN,8.010000e-08,1.970000e-08
2,2023-02-24 11:36:56,24/02/2023,2.349,11:36:56.075,0.0,0.08,0.04,0.12,0.0,0.0,...,-0.0197,-0.015,-0.0251,-0.0148,0.0142,-0.0066,-0.0157,NaN,8.010000e-08,1.970000e-08
3,2023-02-24 11:36:57,24/02/2023,3.517,11:36:57.243,0.0,0.08,0.04,0.12,0.0,0.0,...,-0.0197,-0.015,-0.0251,-0.0148,0.0142,-0.0066,-0.0157,NaN,8.010000e-08,1.970000e-08
4,2023-02-24 11:37:02,24/02/2023,8.837,11:37:02.563,0.0,0.08,0.04,0.12,0.0,0.0,...,0.0018,-0.015,-0.0251,-0.0148,0.0142,-0.0066,-0.0152,NaN,4.200000e-09,1.800000e-09


## Data processing

In [5]:
#print vaclog comments
print(pd.unique(vaclog["Live comments"]))

[nan 'starting injection' 'inj complete']


In [6]:
#print hv comments
print(pd.unique(hv["Comment"]))
hv_comm = hv.loc[:,["datetime","Comment"]]
hv_comm.dropna(inplace=True)
print(hv_comm)
hv_comm.index

[nan 'autozero done' 'start experiment' 'electron activity'
 'grid transparent' 'grid repelling' 'grid transparent 2'
 'grid repelling 2' 'grid transparent 3' 'electron activity off']
                 datetime                Comment
35    2023-02-24 11:37:48          autozero done
36    2023-02-24 11:37:49          autozero done
37    2023-02-24 11:37:50          autozero done
38    2023-02-24 11:37:51          autozero done
39    2023-02-24 11:37:53          autozero done
6033  2023-02-24 13:37:29       start experiment
6034  2023-02-24 13:37:30       start experiment
6035  2023-02-24 13:37:31       start experiment
6036  2023-02-24 13:37:33       start experiment
6074  2023-02-24 13:38:24      electron activity
6075  2023-02-24 13:38:25      electron activity
6076  2023-02-24 13:38:26      electron activity
6077  2023-02-24 13:38:27      electron activity
8598  2023-02-24 14:30:13       grid transparent
8599  2023-02-24 14:30:15       grid transparent
8600  2023-02-24 14:30:16       

Int64Index([   35,    36,    37,    38,    39,  6033,  6034,  6035,  6036,
             6074,  6075,  6076,  6077,  8598,  8599,  8600,  8601, 12849,
            12850, 12851, 12852, 13501, 13502, 13503, 14126, 14127, 14128,
            14129, 14487, 14488, 14489, 14490, 15494, 15495, 15496, 15944,
            15945, 15946, 15947, 15948],
           dtype='int64')

#### CernOx temperature R-T conversion

In [7]:
#Temperature curve for CERNOX - for temp stability
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]

#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


## Checking injection volume pressure drop linearity

In [8]:
#defining a function for the straight line
def fit(x,a,b):
    return a*x + b
#curve fit for the data
y_val = vaclog["injection 1mbar"].values
x_val = (vaclog["Elapsed time"]/3600).values
params, cov = curve_fit(fit,x_val,y_val)
#straight line parameters
a, b = params
print(f"Fitted line: y = {a:.2f}*x+{b:.2f}")

Fitted line: y = -0.00*x+1.10


#### Data preparation for calculating max desorption yield (w/o recycling)

In [9]:
#Filtering background signal

#Find the timestamp where emission was turned on and off
#print("Result 1:",hv.loc[hv["Comment"]=="grid transparent"])
#print("Result 2:", hv.loc[hv["Comment"]=="grid repelling"])
#print("Result 3:", hv.loc[hv["Comment"]=="electron activity grid repelling"])

x=pd.Timestamp(2023, 1, 12, 17 ,23, 58)
y=pd.Timestamp(2023, 1, 12, 17, 28, 43)
z=pd.Timestamp(2023, 1, 12, 17, 20, 13)

#then find the time value closest to it in the MID scan file 
#print("\n","Result 4:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - x).abs().argsort(),:])
#print("\n","Result 5:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - y).abs().argsort(),:])
#print("\n","Result 6:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - z).abs().argsort(),:])

#take a mass with a signal known to be in the electronics background
elec_bck = np.mean(mid["mass 5.5_value"])

#apply elec background correction to h2 data
mid["mass 2, corr"] = mid["mass 2.0_value"] - elec_bck

#apply base pressure correction to mass 2.0 data
dyn_bck = np.mean(mid["mass 2, corr"][12:56])
print("dyn_ bck",f"{dyn_bck}")

#subtracting dyn bck like this deletes useful data
mid["mass 2, corr"] = mid["mass 2, corr"] - dyn_bck

#remove values below 0:
mid["mass 2, corr"].clip(0, inplace=True) 

#From the signal, get the partial pressures 
h2_sens = 2.13e-2 #A/mbar
h2_equiv = 2.49 #H2 equivalent pressures from N2 reading

#account for the thermal transpiration factor
T2 = 294
T1 = 4.2
T_coef = np.sqrt(T2/T1)

#apply correction factors 
p_h2 = (mid["mass 2, corr"][91:136]*h2_equiv)/(h2_sens*T_coef) 
mid.insert(22,"p_h2",p_h2)
#print(p_h2)

#baseline and emission pp for partial pressure delta
em_h2 = mid["p_h2"][91:136] #hardcoded values for "esd on"
base_h2 = mid["p_h2"][12:56] #hardcoded values for "esd off"
print(base_h2)

KeyError: 'mass 5.5_value'

In [ ]:
#NOT USED!

# #non hard coded values for "esd on/off"
# #find the local maxima
# print(mid.p_h2.values)
# mid['ESD max'] = mid.iloc[argrelextrema(mid.p_h2.values, np.greater_equal, order=5)[0]]['p_h2']
# mid['ESD min'] = mid.iloc[argrelextrema(mid.p_h2.values, np.less_equal, order=5)[0]]['p_h2']

# print(mid["ESD max"])
# print(mid["ESD min"])
# #take the 10 next values from max - define as "em on"


# #take the 10 next values from min - define as "em off"

# Apparent desorption yield calculation for H2
### Formula for the cryo setup

$$
\eta_{app}=\frac{C_{comb}\cdot (p_{n,EM} - \overline{p}_{base}) \cdot q_e}{k_B\cdot T \cdot I_{n}}
$$

T here is @RT and therefore thermal transpiration compensation should be applied afterwards to $\Delta p$.

$C_{comb}$ takes into account the conductance from the sample via the copper tube for H2 and the pumping speed for turbo pump

In [ ]:
#calculation of delta_p-s
dp1 = (np.mean(p_h2)-np.mean(base_h2))*100
print(dp1)

#Constants
k_B = scipy.constants.value("Boltzmann constant")
T = 4.2
q_e = scipy.constants.value("elementary charge")
c_h2 = 37.91/1000
s_h2 = 42/1000
x_h2 = 1/((1/c_h2)+(1/s_h2))
print(x_h2)

#Emission current
I_em = np.nanmean(hv["em"].nlargest(30))
print(I_em)


In [ ]:
#Desorption yield
eta_h2 = (x_h2*dp1*q_e)/(k_B*T*I_em)
#eta_co = (c_co*(dp1-dp2)*q_e)/(k_B*T*I_em)
print("The desorption yield for H2 [mol/el]:",eta_h2,"\n",)#""The desorption yield for CO2 [mol/el]:",eta_co2)

#### Data manipulation for point-by-point ESD

In [ ]:
#make a sub-df from mid that includes only mass 2 data
h2 = mid.loc[:, ["mass 2.0_time", "mass 2.0_value"]]
#rename columns
h2 = h2.rename(columns={"mass 2.0_time": "datetime","mass 2.0_value": "Ion current"}).reset_index(drop=True)

print(h2)

In [ ]:
# Convert the 'time' column to datetime index for both dataframes
hv.index = (pd.to_datetime(hv['datetime']).dt.round(freq="S"))
h2.index = (pd.to_datetime(h2['datetime']).dt.round(freq="S"))
print(hv)

# Reindex df1 to match the datetime index of df2
hv = hv.reindex(h2.index, method='nearest', limit=1)

#Drop the "datetime" columns
hv.drop('datetime', axis=1, inplace=True)
h2.drop('datetime', axis=1, inplace=True)

# Now df1 and df2 have the same datetime index and can be joined or aligned
# by joining on the index
hv_aligned = hv.merge(h2,left_index=True, right_index=True, how="inner")

print("aligned",hv_aligned)
print(hv_aligned.describe())

In [ ]:
#Filtering background signal for interpolated datasets

#take a mass with a signal known to be in the background
elec_bck = np.mean(mid["mass 5.5_value"])

#apply base pressure correction to the rest of the mass spectrum
hv_aligned["mass 2 corr"] = hv_aligned["Ion current"]-elec_bck

#From the signal, get the partial pressures 
h2_sens = 2.13e-2 #A/mbar
h2_equiv = 0.38 #H2 equivalent pressures from N2 reading
p_h2_ip = hv_aligned["mass 2 corr"]*h2_equiv/(h2_sens) 

#account for the thermal transpiration factor
T2 = 4.2
T1 = 293
T_coef = np.sqrt(T2/T1)
p_h2_ip = p_h2_ip*T_coef 

#insert partial pressures into the dataframe
hv_aligned.insert(22,"hydrogen pp",p_h2_ip)


In [ ]:
hv_aligned.fillna(0,inplace=True)
hv_aligned["hydrogen pp"].describe()

#### Using polynomial regression, remove the pressure decrease from pumping

In [ ]:
""" # Load the data
t = time["Elapsed time, mass 2"]
print(t.describe())
I_ion = mid["Ion current"]
print(I_ion.describe())
#The function for the ion current-time relationship is not known. 
#numpy method for creating a polynomial model    
model = np.poly1d(np.polyfit(t, I_ion, 8))
print(model(t))

rsquared = r2_score(I_ion,model(t))
print(rsquared)

# Print the original function
plt.figure(figsize=(12,6))
plt.plot(t, I_ion)

# Add labels and title
plt.xlabel('time (s)')
plt.ylabel('Ion current')
plt.title('Ion current vs time')

plt.yscale("log")
plt.show()
 """

In [ ]:
""" #normalise measured data to fit
x = np.linspace(5095.951000, 19700, 3095)
I_norm = I_ion/model(x)

# Plot the normalised ion current
plt.figure(figsize=(12,6))
plt.plot(t, I_norm)

# Add labels and title
plt.xlabel('time (s)')
plt.ylabel('normalised ion current (a.u.)')
plt.title('Ion current vs time')

plt.yscale("linear")
plt.show()

#now we have  """

#### Checking injection volume pressure drop linearity

In [ ]:
#defining a function for the straight line
def fit(x,a,b):
    return a*x + b
#curve fit for the data
y_val = vaclog["injection 1mbar"].values
x_val = (vaclog["Elapsed time"]/3600).values
params, cov = curve_fit(fit,x_val,y_val)
#straight line parameters
a, b = params
print(f"Fitted line: y = {a:.2f}*x+{b:.2f}")

## Desorption yield point-by-point

$$
\eta_{app}=\frac{C_{comb}\cdot (p_{n,EM} - \overline{p}_{base}) \cdot q_e}{k_B\cdot T \cdot I_{n}}
$$

In [ ]:
""" #partial pressure delta
#wrong methodology
p_delta = 
p_delta = p_delta.clip(lower=0)
print(p_delta)
#hv_shifted = hv_aligned.shift(periods=1)
#print(hv_shifted)
#p_delta = hv_aligned["hydrogen pp"] - hv_shifted["hydrogen pp"]
#print(hv_aligned["hydrogen pp"].describe())
#print(hv_shifted["hydrogen pp"].describe())
print(p_delta.describe()) """

In [ ]:
""" #constants are already defined
#emission current and H2 partial pressures are now available at the same sampling rates
print(hv_aligned.describe())
I_n = hv_aligned["em"].replace(0, np.nan).dropna()

I_n.where(hv_aligned["em"] >= 5e-7, np.nan, inplace=True)
#pd.df.where(cond= "Where cond is True, keep the original value. Otherwise, replace with NaN")
print(I_n.describe())

eta = ((x_h2*p_delta*q_e)/(k_B*T*I_n)).dropna()
eta = eta.loc[eta.index[1]:]
print("desorption yields point-by-point:","\n", eta) """

In [ ]:
""" #slicing eta to match hv_aligned["Elapsed time"]:
# Slice x using the index of y
hv_downsampled = hv_aligned.loc[eta.index, :] """

## Electron dose


Find:

$$
Q = \int_{t_1}^{t_2} I \,dt
$$

Where $t_2$ and $t_1$ are the timestamps at which an emission current reading was taken


In [ ]:
""" #Integrating the emission current
def integrand(t, i):
    return i*t

i = I_n.dropna().reset_index() 

#for testing:
#t2_2 = (hv_aligned["Elapsed time"].diff(periods=1).fillna(0))

#adding an elapsed time column
timestamp3=[]
for j in hv["Time"]:
    timestamp3.append(j)
    
#making the column into a datetime object
timestamps3=pd.to_datetime(i["datetime"],format="%H:%M:%S")
runtime3=(timestamps3-timestamps3[0]).dt.total_seconds()    
i.insert(2,"Elapsed time",runtime3)

#define the n, n-1 difference in a variable "t2"
t2 = (i["Elapsed time"].diff(periods=1).fillna(0))

#replace all values from t2 that are further away than 50% of the median of the sampling rate (.. > 9) with the median value
t2.where(t2 <= 12,6,inplace=True)
print(t2)

#integration
ar = []
for x in t2:
    itr = 0
    I = quad(integrand, 0, x, args=(i["em"][itr]))
    ar.append(I[0])
    itr += 1
    #if previous value was not NaN, replace the value with the median of elapsed time
    #if 
     """

In [ ]:
""" #calculate the cumulative charge
Q_cumul = np.cumsum(ar)
Q_sum = sum(ar)
print("Total charge emitted from EGA during ~5000 seconds:", f"{Q_sum:.4E}", "C")

#cumulative nr of electrons emitted
N_e = Q_sum/q_e
Ne_cumul = Q_cumul/q_e
print("Total number of electron emitted during ~5000 seconds:", f"{N_e:.4E}", "e")
      
#To get the electron dose, divide n of electrons by sample surface area of 276 cm2, multiply by grid transmittance of 0.9
tot_dose = (N_e/276)*0.9
cumul_dose = (Ne_cumul/276)*0.9
print("Total electron dose on the sample ~5000 seconds:", f"{tot_dose:.4E}", "e-/cm^2")
      
#add these to the "i" dataframe:
i.insert(3,"Cumulative charge", Q_cumul)
i.insert(4,"Cumulative n. of electrons emitted from EGA", Ne_cumul)
i.insert(5,"Cumulative electron dose on the HL sample", cumul_dose)
i.set_index("datetime", drop=True, inplace=True)

#Ignore the first value, also, match the shape of eta
i = i.loc[i.index[1]:]
print(i)
 """

# Plotting

In [ ]:
#define parameters for plotting
#annotation params
font = dict(size = "x-small", color ="green", style ="italic",rotation="vertical")

plt.rcParams["figure.figsize"] = [8,5]
plt.rcParams["axes.edgecolor"] = "black"
plt.rcParams["axes.grid"] = True
plt.rcParams["grid.color"] = "black"
plt.rcParams["grid.linewidth"] = 0.35

formatter = md.DateFormatter('%H:%M')

# Create the "graphs" folder path
graphs_folder = os.path.join(os.getcwd(), 'graphs')

# Ensure the "graphs" folder exists, if not create it
if not os.path.exists(graphs_folder):
    os.makedirs(graphs_folder)
    

In [ ]:
#plot the aligned dataframe results
plt.figure(figsize=(12,6))
plt.plot(hv_aligned["Elapsed time"],hv_aligned["Ion current"],linestyle="-",marker=".", markersize=4,label='H2 ion current')
plt.plot(hv_aligned["Elapsed time"],hv_aligned["em"],linestyle="-",marker=".", markersize=4,label='Emission current')
plt.xlabel('Elapsed time (s)')
plt.ylabel('Current (A)')
plt.legend(loc="upper right")
plt.title("Current evolution 1 ML tech. surface pre-injection, EGA fully in, dynamic vacuum")


plt.yscale('log')


In [ ]:
#plot the h2 mid 
#seems to be too little rows?
plt.figure(figsize=(12,6))
plt.plot(hv_aligned["Elapsed time"],hv_aligned["Ion current"],linestyle="-",marker=".", markersize=4,label='H2 ion current')
plt.xlabel('Elapsed time (s)')
plt.ylabel('Ion current (A)')
plt.legend(loc="upper right")
plt.title("Ion current evolution of H2, 1 ML tech. surface pre-injection, EGA fully in, dynamic vacuum")


plt.yscale('linear')


In [ ]:
#Plotting the all the relevant MID spectrums
plt.figure(figsize=(14,8))

plt.plot(new_df["Timestamp, mass 2"],mid["mass 2.0_value"]/2.13e-2,marker=".", markersize=4,label='M2')# MBAR
# plt.plot(new_df["Timestamp, mass 44"],mid["mass 44.0_value"],marker=".", markersize=4,label='M44')

plt.xlabel('Timestamp')
plt.ylabel('Pressure (mbar)')
plt.legend(title="Mol masses", loc="upper right")
plt.title("S1 pressure from MID scan")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(14))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')

#write comments
trans  = transforms.blended_transform_factory(
    ax.transData, ax.transAxes)
for i in hv_comm.index:
    plt.text(hv_comm["datetime"][i],0.95,str(hv_comm["Comment"][i]),fontdict=font, transform=trans)


plt.savefig(r'./graphs/pressure from mid.png')
plt.show()

In [ ]:
#plotting HiVolta log
plt.figure(figsize=(12,6))
plt.plot(hv["Elapsed time"],hv["hv_grid"],marker=".", markersize=4,label='Grid current')
plt.plot(hv["Elapsed time"],hv["em"],marker=".", markersize=4,label='Emission current')
plt.xlabel('Elapsed time (s)')
plt.ylabel('Current (A)')
plt.legend(loc="upper right")
plt.title("Current evolution 1 ML tech. surface pre-injection, EGA fully in, dynamic vacuum")


plt.yscale('log')


In [ ]:
#Plotting BA2, BA1 pressure
plt.figure(figsize=(12,6))
plt.plot(vaclog["Time"],vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2')
plt.plot(vaclog["Time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')
plt.xlabel('Timestamp')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("S1 Total pressure evolution")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')    
       
plt.savefig(r'./graphs/Barions.png')
plt.show()

In [ ]:
## Plotting the temp evolution
plt.figure(figsize=(10,6))
plt.plot((vaclog["Time"]),polyfit(A,vaclog["T-CERNOX"]),marker=".", markersize=5,label='T-CERNOX')          
plt.xlabel('Timestamp')
plt.ylabel('Temperature (K)')
plt.legend()
plt.title("S1 Temperature evolution")
plt.yscale('linear')
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')
    
plt.savefig(r'./graphs/Temp.png')
plt.show()

In [ ]:
#checking inj pressure linearity, graph
fig, ax = plt.subplots()
y_fit = fit(x_val,a,b)
ax.plot((vaclog["Elapsed time"]/3600),vaclog["injection 1mbar"],marker=".", markersize=5,label='inj volume pressure')
ax.plot((vaclog["Elapsed time"]/3600),y_fit, "--",color="red", label="Y= -ax + b fit") 
ax.legend(loc="upper right")

ax.set_xlabel('Elapsed time (h)')
ax.set_ylabel('Pressure (mbar)')
ax.set_yscale('linear')
plt.title("Injection pressure evolution 1mbar conductance gauge")

In [ ]:
#plotting point-by-point eta
plt.figure(figsize=(10,6))
plt.plot(hv_downsampled["Elapsed time"],eta,marker=".", markersize=4,label='eta')
#plt.plot(vaclog["Elapsed time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Electon stimulated desorption yield (M/el)')
plt.legend()
plt.title("ESD vs time, p_n - p_n-5")
ax = plt.gca()
ax.set_yscale('log')
ax.grid(which="major",axis="x")
plt.tick_params(axis="y", which='minor')
plt.grid(visible=None, which='both', axis='both')
plt.show()

In [ ]:
#plotting esd vs electron dose
plt.figure(figsize=(10,6))
plt.plot(i["Cumulative electron dose on the HL sample"],eta,marker=".", markersize=4,label='eta')
#plt.plot(vaclog["Elapsed time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')

plt.xlabel('Electron dose (e/cm^2)')
plt.ylabel('Electon stimulated desorption yield (M/el)')
plt.legend()
plt.title("ESD vs dose")
ax = plt.gca()
ax.set_yscale('log')
ax.grid(which="major",axis="x")
plt.tick_params(axis="y", which='minor')
plt.grid(visible=None, which='both', axis='both')
plt.show()